# TensorBoard：训练过程可视化 
https://tf.wiki/zh/basic/tools.html

实例：查看多层感知机模型的训练情况 
最后提供一个实例，以前章的 多层感知机模型 为例展示 TensorBoard 的使用：

In [1]:
import tensorflow as tf
import numpy as np
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
        return self.train_data[index, :], self.train_label[index]
    
    
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

In [3]:
num_batches = 1000
batch_size = 50
learning_rate = 0.001
log_dir = 'tensorboard'
model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
summary_writer = tf.summary.create_file_writer(log_dir)     # 实例化记录器
tf.summary.trace_on(profiler=True)  # 开启Trace（可选）
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
        with summary_writer.as_default():                           # 指定记录器
            tf.summary.scalar("loss", loss, step=batch_index)       # 将当前损失函数的值写入记录器
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
with summary_writer.as_default():
    tf.summary.trace_export(name="model_trace", step=0, profiler_outdir=log_dir)    # 保存Trace信息到文件（可选）

batch 0: loss 2.440863
batch 1: loss 2.351085
batch 2: loss 2.354211
batch 3: loss 2.186523
batch 4: loss 2.132562
batch 5: loss 2.077986
batch 6: loss 1.987841
batch 7: loss 2.004390
batch 8: loss 1.803120
batch 9: loss 1.756266
batch 10: loss 1.730825
batch 11: loss 1.706142
batch 12: loss 1.720908
batch 13: loss 1.661014
batch 14: loss 1.520746
batch 15: loss 1.509544
batch 16: loss 1.407559
batch 17: loss 1.490591
batch 18: loss 1.297632
batch 19: loss 1.172279
batch 20: loss 1.151317
batch 21: loss 1.183685
batch 22: loss 1.186458
batch 23: loss 1.155329
batch 24: loss 1.314043
batch 25: loss 1.078335
batch 26: loss 1.080948
batch 27: loss 0.929244
batch 28: loss 0.936921
batch 29: loss 1.106667
batch 30: loss 0.782553
batch 31: loss 0.997983
batch 32: loss 0.935974
batch 33: loss 0.998367
batch 34: loss 1.074695
batch 35: loss 0.871235
batch 36: loss 0.990025
batch 37: loss 0.799476
batch 38: loss 1.011181
batch 39: loss 0.895881
batch 40: loss 0.652561
batch 41: loss 0.894675
ba

batch 355: loss 0.379764
batch 356: loss 0.395259
batch 357: loss 0.187741
batch 358: loss 0.163589
batch 359: loss 0.276098
batch 360: loss 0.289661
batch 361: loss 0.257722
batch 362: loss 0.230889
batch 363: loss 0.313603
batch 364: loss 0.374112
batch 365: loss 0.389441
batch 366: loss 0.198932
batch 367: loss 0.330717
batch 368: loss 0.170879
batch 369: loss 0.275873
batch 370: loss 0.221428
batch 371: loss 0.193662
batch 372: loss 0.309786
batch 373: loss 0.519774
batch 374: loss 0.258954
batch 375: loss 0.188091
batch 376: loss 0.501827
batch 377: loss 0.377193
batch 378: loss 0.225625
batch 379: loss 0.170722
batch 380: loss 0.213849
batch 381: loss 0.335915
batch 382: loss 0.204249
batch 383: loss 0.225048
batch 384: loss 0.209855
batch 385: loss 0.529824
batch 386: loss 0.266728
batch 387: loss 0.127914
batch 388: loss 0.264822
batch 389: loss 0.251116
batch 390: loss 0.336221
batch 391: loss 0.162031
batch 392: loss 0.174867
batch 393: loss 0.127399
batch 394: loss 0.394436


batch 710: loss 0.264258
batch 711: loss 0.103257
batch 712: loss 0.226038
batch 713: loss 0.201210
batch 714: loss 0.197025
batch 715: loss 0.249662
batch 716: loss 0.341942
batch 717: loss 0.168882
batch 718: loss 0.265138
batch 719: loss 0.189167
batch 720: loss 0.158369
batch 721: loss 0.306120
batch 722: loss 0.176407
batch 723: loss 0.246556
batch 724: loss 0.421845
batch 725: loss 0.306097
batch 726: loss 0.331393
batch 727: loss 0.229816
batch 728: loss 0.250017
batch 729: loss 0.146112
batch 730: loss 0.106469
batch 731: loss 0.229679
batch 732: loss 0.204585
batch 733: loss 0.276972
batch 734: loss 0.182763
batch 735: loss 0.263279
batch 736: loss 0.208866
batch 737: loss 0.159641
batch 738: loss 0.166756
batch 739: loss 0.075537
batch 740: loss 0.073339
batch 741: loss 0.101506
batch 742: loss 0.275496
batch 743: loss 0.232008
batch 744: loss 0.157162
batch 745: loss 0.095400
batch 746: loss 0.114575
batch 747: loss 0.407539
batch 748: loss 0.126552
batch 749: loss 0.147467
